## Step 1: Import Libraries

In [5]:
import cv2
import os
import time
from threading import Thread

## Step 3: Process Video Frames

In [6]:
# Function to run the countdown in a separate thread
def countdown_thread(countdown_dict):
    while countdown_dict['value'] > 0:
        time.sleep(1)
        countdown_dict['value'] -= 1

In [7]:
def capture_video(category, num_recordings, output_dir, video_url):

    tutorial_video = cv2.VideoCapture(video_url)
    while True:
        ret, frame = tutorial_video.read()
        if not ret:
            tutorial_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue
        cv2.putText(frame, 'Press C to continue', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Tutorial', frame)
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break
    tutorial_video.release()
    cv2.destroyWindow('Tutorial')

    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    # retrieve properties of the capture object
    cap_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    cap_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    cap_fps = cap.get(cv2.CAP_PROP_FPS)
    print('* Capture width:', cap_width)
    print('* Capture height:', cap_height)
    print('* Capture FPS:', cap_fps)
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    recording = False
    recording_counter = 0
    countdown = 2
    countdown_dict = {'value': countdown}  # Countdown before starting recording
    recording_frame_counter = 0
    
    while recording_counter < num_recordings:
        ret, frame = cap.read()
        
        # Create a separate frame for display purposes
        display_frame = frame.copy()
        display_frame = cv2.flip(display_frame, 1)

        cv2.putText(display_frame, f'Count: {recording_counter}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 221, 0), 2, cv2.LINE_AA)
        cv2.putText(display_frame, f'Frame: {recording_frame_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 221, 0), 2, cv2.LINE_AA)
        if not recording:
            if countdown_dict['value'] > 0:
                cv2.putText(display_frame, f'Starting in {countdown_dict["value"]}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if not hasattr(capture_video, 'countdown_thread') or not capture_video.countdown_thread.is_alive():
                    capture_video.countdown_thread = Thread(target=countdown_thread, args=(countdown_dict,))
                    capture_video.countdown_thread.start()
            else:
                recording = True
                countdown_dict['value'] = countdown
                out = cv2.VideoWriter(os.path.join(output_dir, f'{category}_{recording_counter}.mp4'), cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 30, (640, 480))
        else:
            if recording_frame_counter < 30:
                recording_frame_counter += 1
                out.write(frame)
                cv2.putText(display_frame, 'Recording', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                recording = False
                recording_counter += 1
                recording_frame_counter = 0
                out.release()
        
        cv2.imshow('Webcam', display_frame)  # Display the frame with overlays
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

## Step 4: Main

In [8]:
# Read the categories and video URLs from the text file
categories = []
video_urls = []
with open('examples/common_words_with_videos.txt', 'r') as file:
    for line in file:
        category, url = line.strip().split(': ')
        categories.append(category)
        video_urls.append(url)

# User inputs
print("Select a category:")
for i, category in enumerate(categories):
    print(f"{i + 1}. {category}")
categories_idx = input("Enter the number of the category: ")

num_recordings = 50

for category_idx in categories_idx.split(","):
    category_idx = int(category_idx) - 1
    selected_category = categories[category_idx]
    selected_video_url = video_urls[category_idx]

    output_dir = f'data/recordings/{selected_category}'
    
    capture_video(selected_category, num_recordings, output_dir, selected_video_url)

print("Recording complete.")

Select a category:
1. Me-
2. Pe-
3. Kenal
4. Kan-
5. Nama
6. Saya
7. A
8. N
9. S
10. Mari
11. Kami
12. Tunjuk
13. -Kan
14. Kepada
15. Anda
16. Proyek
* Capture width: 640.0
* Capture height: 480.0
* Capture FPS: 30.0


NameError: name 'holistic' is not defined